In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [30]:
#use requests to get the data and then use beautiful soup so we can parse through it
oscars2019_raw = requests.get('https://www.oscars.org/oscars/ceremonies/2019').text

oscars2019_bs = BeautifulSoup(oscars2019_raw)

In [31]:
#get all award groups
oscars2019_groups = oscars2019_bs.find_all('div',{'class':'view-grouping'})

#look at the first one
oscars2019_groups[0]

<div class="view-grouping"><div class="view-grouping-header"><h2>Actor in a Leading Role</h2></div><div class="view-grouping-content"> <h3><span class="golden-text">Winner</span></h3>
<div class="views-row views-row-1 views-row-odd views-row-first views-row-last">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Rami Malek</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Bohemian Rhapsody
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<h3>Nominees</h3>
<div class="views-row views-row-1 views-row-odd views-row-first">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Christian Bale</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Vice
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<div class="views-row views-row-2 views-row-ev

In [32]:
#look at the last one
oscars2019_groups[-1]

<div class="view-grouping"><div class="view-grouping-header">The Wife</div><span class="label">1 Nomination</span><div class="view-grouping-content"> <div>
<div class="views-field views-field-title-field"> <div class="field-content">Actress in a Leading Role - Glenn Close</div> </div> </div>
</div></div>

In [33]:
#count number of award groups
len(oscars2019_groups)

104

In [34]:
#HTML tree to find more specific parents
oscars2019_parent_group = oscars2019_bs.find_all('div',{'id':'quicktabs-tabpage-honorees-0'})

#count groups matching the pattern; only 1
len(oscars2019_parent_group)

1

In [35]:
#search within the parent group
oscars2019_true_groups = oscars2019_parent_group[0].find_all('div',{'class':'view-grouping'})
#number of award groups
len(oscars2019_true_groups)

24

In [36]:
#loop to find award names in each group
for group in oscars2019_parent_group[0].find_all('div',{'class':'view-grouping-header'}):
    print(group.text)

Actor in a Leading Role
Actor in a Supporting Role
Actress in a Leading Role
Actress in a Supporting Role
Animated Feature Film
Cinematography
Costume Design
Directing
Documentary (Feature)
Documentary (Short Subject)
Film Editing
Foreign Language Film
Makeup and Hairstyling
Music (Original Score)
Music (Original Song)
Best Picture
Production Design
Short Film (Animated)
Short Film (Live Action)
Sound Editing
Sound Mixing
Visual Effects
Writing (Adapted Screenplay)
Writing (Original Screenplay)


In [37]:
#find tag for "Actor in a Leading Role"
oscars2019_groups[0]

<div class="view-grouping"><div class="view-grouping-header"><h2>Actor in a Leading Role</h2></div><div class="view-grouping-content"> <h3><span class="golden-text">Winner</span></h3>
<div class="views-row views-row-1 views-row-odd views-row-first views-row-last">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Rami Malek</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Bohemian Rhapsody
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<h3>Nominees</h3>
<div class="views-row views-row-1 views-row-odd views-row-first">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Christian Bale</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Vice
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<div class="views-row views-row-2 views-row-ev

In [38]:
#get parent
oscars2019_group0_parent = oscars2019_groups[0].parent

#find relevant children
oscars2019_group0_parent_children = oscars2019_group0_parent.find_all('div',{'class':'view-grouping'})

#count
len(oscars2019_group0_parent_children)

24

Look at relevant fields and output HTML output for content mapping

In [39]:
oscars2019_true_groups[0]

<div class="view-grouping"><div class="view-grouping-header"><h2>Actor in a Leading Role</h2></div><div class="view-grouping-content"> <h3><span class="golden-text">Winner</span></h3>
<div class="views-row views-row-1 views-row-odd views-row-first views-row-last">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Rami Malek</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Bohemian Rhapsody
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<h3>Nominees</h3>
<div class="views-row views-row-1 views-row-odd views-row-first">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Christian Bale</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Vice
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<div class="views-row views-row-2 views-row-ev

In [40]:
oscars2019_true_groups[0].find_all('div',{'class':"views-field views-field-field-actor-name"})

[<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Rami Malek</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">Christian Bale</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">Bradley Cooper</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">Willem Dafoe</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">Viggo Mortensen</h4> </div>]

In [41]:
oscars2019_true_groups[0].find_all('h4')

[<h4 class="field-content">Rami Malek</h4>,
 <h4 class="field-content">Christian Bale</h4>,
 <h4 class="field-content">Bradley Cooper</h4>,
 <h4 class="field-content">Willem Dafoe</h4>,
 <h4 class="field-content">Viggo Mortensen</h4>]

In [42]:
oscars2019_true_groups[0].find_all('div',{'class':"views-field views-field-title"})

[<div class="views-field views-field-title"> <span class="field-content">Bohemian Rhapsody
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">Vice
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">A Star Is Born
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">At Eternity's Gate
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">Green Book
 </span> </div>]

In [43]:
oscars2019_true_groups[0].find_all('span')

[<span class="golden-text">Winner</span>,
 <span class="field-content">Bohemian Rhapsody
 </span>,
 <span class="field-content"></span>,
 <span class="field-content">Vice
 </span>,
 <span class="field-content"></span>,
 <span class="field-content">A Star Is Born
 </span>,
 <span class="field-content"></span>,
 <span class="field-content">At Eternity's Gate
 </span>,
 <span class="field-content"></span>,
 <span class="field-content">Green Book
 </span>,
 <span class="field-content"></span>]

In [44]:
oscars2019_true_groups[-1].find_all('div',{'class':"views-field views-field-field-actor-name"})

[<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Green Book</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">The Favourite</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">First Reformed</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">Roma</h4> </div>,
 <div class="views-field views-field-field-actor-name"> <h4 class="field-content">Vice</h4> </div>]

In [45]:
oscars2019_true_groups[-1].find_all('div',{'class':"views-field views-field-title"})

[<div class="views-field views-field-title"> <span class="field-content">Written by Nick Vallelonga, Brian Currie, Peter Farrelly
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">Written by Deborah Davis and Tony McNamara
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">Written by Paul Schrader
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">Written by Alfonso Cuarón
 </span> </div>,
 <div class="views-field views-field-title"> <span class="field-content">Written by Adam McKay
 </span> </div>]

In [46]:
oscars2019_true_groups[0]

<div class="view-grouping"><div class="view-grouping-header"><h2>Actor in a Leading Role</h2></div><div class="view-grouping-content"> <h3><span class="golden-text">Winner</span></h3>
<div class="views-row views-row-1 views-row-odd views-row-first views-row-last">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Rami Malek</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Bohemian Rhapsody
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<h3>Nominees</h3>
<div class="views-row views-row-1 views-row-odd views-row-first">
<div class="views-field views-field-field-actor-name"> <h4 class="field-content">Christian Bale</h4> </div>
<div class="views-field views-field-title"> <span class="field-content">Vice
</span> </div>
<div class="views-field views-field-edit-node"> <span class="field-content"></span> </div> </div>
<div class="views-row views-row-2 views-row-ev

In [47]:
category = oscars2019_true_groups[0].find_all('h2')[0].text
print("The name of the category is:",category)

The name of the category is: Actor in a Leading Role


In [48]:
names = []
for _nominee in oscars2019_true_groups[0].find_all('h4'):
    nominee_name = _nominee.text
    names.append(nominee_name)
    print("The name of a nominee is:",nominee_name)

The name of a nominee is: Rami Malek
The name of a nominee is: Christian Bale
The name of a nominee is: Bradley Cooper
The name of a nominee is: Willem Dafoe
The name of a nominee is: Viggo Mortensen


In [49]:
movies = []

for _movie in oscars2019_true_groups[0].find_all('span'):
    if len(_movie.text) > 0:
        movie_name = _movie.text.strip()
        movies.append(movie_name)
        print("The name of a movie is:",movie_name)

The name of a movie is: Winner
The name of a movie is: Bohemian Rhapsody
The name of a movie is: Vice
The name of a movie is: A Star Is Born
The name of a movie is: At Eternity's Gate
The name of a movie is: Green Book


In [50]:
#extract relevant info from subdivs
actor_nominees = oscars2019_true_groups[0].find_all('div',{'class':'views-row'})

for i,nominee in enumerate(actor_nominees):
    
    if i == 0:
        winner = 'Won'
    else:
        winner = 'Lost'
    
    #list of all subdivs
    subdivs = nominee.find_all('div')
    
    #first subdiv (for an actor) is the name
    name = subdivs[0].text.strip()
    
    #second subdiv (for an actor) is the movie name
    movie = subdivs[1].text.strip()
    
    print("{0} was nominated for \"{1}\" and {2}.".format(name,movie,winner))

Rami Malek was nominated for "Bohemian Rhapsody" and Won.
Christian Bale was nominated for "Vice" and Lost.
Bradley Cooper was nominated for "A Star Is Born" and Lost.
Willem Dafoe was nominated for "At Eternity's Gate" and Lost.
Viggo Mortensen was nominated for "Green Book" and Lost.


In [51]:
#reversing 'movie' and 'name' works for another award category (making sure code is sound)
original_screenplay_nominees = oscars2019_true_groups[-1].find_all('div',{'class':'views-row'})

for i,nominee in enumerate(original_screenplay_nominees):
    if i == 0:
        winner = 'Won'
    else:
        winner = 'Lost'
        
    subdivs = nominee.find_all('div')
    
    #movie and name reversed
    movie = subdivs[0].text.strip()
    name = subdivs[1].text.strip()
    
    print("{0} was nominated for \"{1}\" and {2}.".format(name,movie,winner))

Written by Nick Vallelonga, Brian Currie, Peter Farrelly was nominated for "Green Book" and Won.
Written by Deborah Davis and Tony McNamara was nominated for "The Favourite" and Lost.
Written by Paul Schrader was nominated for "First Reformed" and Lost.
Written by Alfonso Cuarón was nominated for "Roma" and Lost.
Written by Adam McKay was nominated for "Vice" and Lost.


In [52]:
#new format of 'name' 'category' 'movie' and 'winner'
for group in oscars2019_true_groups:
    category = group.find_all('h2')[0].text
    
    for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
        if i == 0:
            winner = 'Won'
        else:
            winner = 'Lost'

        subdivs = nominee.find_all('div')
        name = subdivs[0].text.strip()
        movie = subdivs[1].text.strip()

        print("{0} was nominated in {1} for {2}\" and {3}.".format(name,category,movie,winner))

Rami Malek was nominated in Actor in a Leading Role for Bohemian Rhapsody" and Won.
Christian Bale was nominated in Actor in a Leading Role for Vice" and Lost.
Bradley Cooper was nominated in Actor in a Leading Role for A Star Is Born" and Lost.
Willem Dafoe was nominated in Actor in a Leading Role for At Eternity's Gate" and Lost.
Viggo Mortensen was nominated in Actor in a Leading Role for Green Book" and Lost.
Mahershala Ali was nominated in Actor in a Supporting Role for Green Book" and Won.
Adam Driver was nominated in Actor in a Supporting Role for BlacKkKlansman" and Lost.
Sam Elliott was nominated in Actor in a Supporting Role for A Star Is Born" and Lost.
Richard E. Grant was nominated in Actor in a Supporting Role for Can You Ever Forgive Me?" and Lost.
Sam Rockwell was nominated in Actor in a Supporting Role for Vice" and Lost.
Olivia Colman was nominated in Actress in a Leading Role for The Favourite" and Won.
Yalitza Aparicio was nominated in Actress in a Leading Role for 

In [53]:
for group in oscars2019_true_groups:
    category = group.find_all('h2')[0].text
    
    if 'Actor' in category or 'Actress' in category:
    
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            name = subdivs[0].text.strip()
            movie = subdivs[1].text.strip()

            print("{0} was nominated in {1} for \"{2}\" and {3}.".format(name,category,movie,winner))
            
    else:
        
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            movie = subdivs[0].text.strip()
            name = subdivs[1].text.strip()

            print("\"{0}\" was nominated in {1} for {2} and {3}.".format(name,category,movie,winner))

Rami Malek was nominated in Actor in a Leading Role for "Bohemian Rhapsody" and Won.
Christian Bale was nominated in Actor in a Leading Role for "Vice" and Lost.
Bradley Cooper was nominated in Actor in a Leading Role for "A Star Is Born" and Lost.
Willem Dafoe was nominated in Actor in a Leading Role for "At Eternity's Gate" and Lost.
Viggo Mortensen was nominated in Actor in a Leading Role for "Green Book" and Lost.
Mahershala Ali was nominated in Actor in a Supporting Role for "Green Book" and Won.
Adam Driver was nominated in Actor in a Supporting Role for "BlacKkKlansman" and Lost.
Sam Elliott was nominated in Actor in a Supporting Role for "A Star Is Born" and Lost.
Richard E. Grant was nominated in Actor in a Supporting Role for "Can You Ever Forgive Me?" and Lost.
Sam Rockwell was nominated in Actor in a Supporting Role for "Vice" and Lost.
Olivia Colman was nominated in Actress in a Leading Role for "The Favourite" and Won.
Yalitza Aparicio was nominated in Actress in a Leadin

In [54]:
#store info in 'nominees_2019' to turn into a dataframe
nominees_2019 = []

for group in oscars2019_true_groups:
    category = group.find_all('h2')[0].text
    
    if 'Actor' in category or 'Actress' in category:
    
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            name = subdivs[0].text.strip()
            movie = subdivs[1].text.strip()
            
            nominee_payload = {'Category':category,
                               'Name':name,
                               'Movie':movie,
                               'Year':2019, 
                               'Winner':winner}
            
            nominees_2019.append(nominee_payload)
            
    else:
        
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            movie = subdivs[0].text.strip()
            name = subdivs[1].text.strip()

            nominee_payload = {'Category':category,
                               'Name':name,
                               'Movie':movie,
                               'Year':2018,
                               'Winner':winner}
            
            nominees_2019.append(nominee_payload)

In [55]:
#pandas dataframe of 2019 nominees
nominees_df = pd.DataFrame(nominees_2019)
nominees_df.head(10)

,Category,Name,Movie,Year,Winner
0,Actor in a Leading Role,Rami Malek,Bohemian Rhapsody,2019,Won
1,Actor in a Leading Role,Christian Bale,Vice,2019,Lost
2,Actor in a Leading Role,Bradley Cooper,A Star Is Born,2019,Lost
3,Actor in a Leading Role,Willem Dafoe,At Eternity's Gate,2019,Lost
4,Actor in a Leading Role,Viggo Mortensen,Green Book,2019,Lost
5,Actor in a Supporting Role,Mahershala Ali,Green Book,2019,Won
6,Actor in a Supporting Role,Adam Driver,BlacKkKlansman,2019,Lost
7,Actor in a Supporting Role,Sam Elliott,A Star Is Born,2019,Lost
8,Actor in a Supporting Role,Richard E. Grant,Can You Ever Forgive Me?,2019,Lost
9,Actor in a Supporting Role,Sam Rockwell,Vice,2019,Lost


In [56]:
nominees_df.to_excel(r'C:\Users\Shailaja\Documents\college`\2019 spring\ban 172 - web analytics\final project\oscars_allyr.xlsx')

FileCreateError: [Errno 2] No such file or directory: 'C:\\Users\\Shailaja\\Documents\\college`\\2019 spring\\ban 172 - web analytics\\final project\\oscars_allyr.xlsx'

In [57]:
#turn parser into a function to apply to other years' nominees
def parse_nominees(true_groups,year):
    nominees_list = []

    for group in true_groups:
        category = group.find_all('h2')[0].text

        if 'Actor' in category or 'Actress' in category:

            for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
                if i == 0:
                    winner = 'Won'
                else:
                    winner = 'Lost'

                subdivs = nominee.find_all('div')
                name = subdivs[0].text.strip()
                movie = subdivs[1].text.strip()
                
                nominee_payload = {'Category':category,
                                   'Name':name,
                                   'Movie':movie,
                                   'Year':year,
                                   'Winner':winner}

                nominees_list.append(nominee_payload)

        else:

            for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
                if i == 0:
                    winner = 'Won'
                else:
                    winner = 'Lost'

                subdivs = nominee.find_all('div')
                movie = subdivs[0].text.strip()
                name = subdivs[1].text.strip()

                nominee_payload = {'Category':category,
                                   'Name':name,
                                   'Movie':movie,
                                   'Year':year,
                                   'Winner':winner}

                nominees_list.append(nominee_payload)
    
    return nominees_list

In [58]:
#use this by including 'sleep' between each request; doesn't overwhlem the server (wasn't working without this)
from time import sleep

In [59]:
print("The start of something.")
sleep(3)
print("The end of something.")

The start of something.
The end of something.


In [60]:
#prints award pages from 2010-2020; 1 second request between URL retrieval
for year in range(2010,2020):
    sleep(1)
    print('https://www.oscars.org/oscars/ceremonies/{0}'.format(year))

https://www.oscars.org/oscars/ceremonies/2010
https://www.oscars.org/oscars/ceremonies/2011
https://www.oscars.org/oscars/ceremonies/2012
https://www.oscars.org/oscars/ceremonies/2013
https://www.oscars.org/oscars/ceremonies/2014
https://www.oscars.org/oscars/ceremonies/2015
https://www.oscars.org/oscars/ceremonies/2016
https://www.oscars.org/oscars/ceremonies/2017
https://www.oscars.org/oscars/ceremonies/2018
https://www.oscars.org/oscars/ceremonies/2019


In [61]:
#create an empty list to store the data we get
all_years_nominees = dict()

for year in range(2010,2020):

    sleep(1) 
    
    #get raw HTML & use beautiful soup
    year_raw_html = requests.get('https://www.oscars.org/oscars/ceremonies/{0}'.format(year)).text
    year_soup_html = BeautifulSoup(year_raw_html)
    
    #get the parent group
    year_parent_group = year_soup_html.find_all('div',{'id':'quicktabs-tabpage-honorees-0'})
    
    #get the true groups under the parent group
    year_true_groups = year_parent_group[0].find_all('div',{'class':'view-grouping'})
    
    #use parsing function
    year_nominees = parse_nominees(year_true_groups,year)
    
    #convert the year_nominees to a df and add them to all_years_nominees
    all_years_nominees[year] = pd.DataFrame(year_nominees)

In [62]:
#dataframe including all years
all_years_nominees_df = pd.concat(all_years_nominees)
all_years_nominees_df.reset_index(drop=True)

,Category,Name,Movie,Year,Winner
0,Actor in a Leading Role,Jeff Bridges,Crazy Heart,2010,Won
1,Actor in a Leading Role,George Clooney,Up in the Air,2010,Lost
2,Actor in a Leading Role,Colin Firth,A Single Man,2010,Lost
3,Actor in a Leading Role,Morgan Freeman,Invictus,2010,Lost
4,Actor in a Leading Role,Jeremy Renner,The Hurt Locker,2010,Lost
...,...,...,...,...,...
1208,Writing (Original Screenplay),"Written by Nick Vallelonga, Brian Currie, Pete...",Green Book,2019,Won
1209,Writing (Original Screenplay),Written by Deborah Davis and Tony McNamara,The Favourite,2019,Lost
1210,Writing (Original Screenplay),Written by Paul Schrader,First Reformed,2019,Lost
1211,Writing (Original Screenplay),Written by Alfonso Cuarón,Roma,2019,Lost


In [63]:
all_years_nominees_df.to_excel(r'C:\Users\Shailaja\Documents\college\2019 spring\ban 172 - web analytics\final project\oscars_allyears.xlsx')